## Content Based Router

This notebook demonstrates using `FunctionalTransformer` with a `RoutingRule` to redirect a message based on the message content.

The Message is routed based on the partial `MessageRoutable` returned from JSONata expression, combining it with default values.

These are the fields in MessageRoutable:

- **topics** (Union[str, List[str]]): The topics to which the message should be published.
- **recipient_list** (List[AgentTag]): List of agents tagged in the message.
- **payload** (JsonDict): The actual content or payload of the message.
- **format** (str): The type of the message.
- **forward_header** (Optional[ForwardHeader]): The header for a forwarded message.
- **context** (Optional[JsonDict]): The context of the message.
- **enrich_with_history** (Optional[int]): The number of previous messages to include in the context.

In this example, we route if based on `routing_key` in the payload.


[ProbeAgent] --(`{"routing_key": "topic_1"}`)-> (default_topic) -> [Routing Agent/RoutingAgent:wire_message] -> (**topic_1**)

[ProbeAgent] --(`{"routing_key": "topic_2"}`)-> (default_topic) -> [Routing Agent/RoutingAgent:wire_message] -> (**topic_2**)


In [ ]:
from pydantic import BaseModel


class CBRMessage(BaseModel):
    routing_key: str
    body: str

In [ ]:
from rustic_ai.core.agents.eip.basic_wiring_agent import BasicWiringAgent
from rustic_ai.core.guild.builders import AgentBuilder, GuildBuilder, RouteBuilder
from rustic_ai.core.guild.dsl import GuildTopics
from rustic_ai.core.utils import jx
from rustic_ai.core.utils.basic_class_utils import get_qualified_class_name
from rustic_ai.core.utils.jexpr import JObj, JxScript

guild_default_topic = GuildTopics.DEFAULT_TOPICS[0]


routing_agent = (
    AgentBuilder(BasicWiringAgent)
    .set_id("RoutingAgent")
    .set_name("Routing Agent")
    .set_description("A content based router that routes messages based on their routing_key.")
    .build_spec()
)

# Create a JxScript to route message based on content
routing_script = JxScript(
    JObj(
        {
            "topics": jx.JExpr("payload").routing_key,
        }
    )
)

# Create the Routing Rule for CBRMessage
cbr_routing_rule = (
    RouteBuilder(routing_agent)
    .filter_on_origin(origin_message_format=get_qualified_class_name(CBRMessage))
    .set_functional_transformer(routing_script)
    .build()
)

# Create the Guild with the Routing Agent and the Routing Rules
guild = (
    GuildBuilder(
        guild_id="MessageRouterGuild",
        guild_name="Message Router Guild",
        guild_description="A guild that routes messages based on their type.",
    )
    .add_agent_spec(routing_agent)
    .add_route(cbr_routing_rule)
    .launch("myorg")
)

In [ ]:
from rustic_ai.core.agents.testutils.probe_agent import ProbeAgent

probe_spec = (
    AgentBuilder(ProbeAgent)
    .set_id("ProbeAgent")
    .set_name("Probe Agent")
    .set_description("A probe agent to test the routing of messages.")
    .add_additional_topic("topic_1")
    .add_additional_topic("topic_2")
    .build_spec()
)

probe_agent: ProbeAgent = guild._add_local_agent(probe_spec)  # type: ignore

In [ ]:
probe_agent.publish_with_guild_route(
    payload=CBRMessage(routing_key="topic_1", body="This is a message for topic 1"),
    topic=guild_default_topic,
)

In [ ]:
probe_agent.print_message_history()

In [ ]:
probe_agent.publish_with_guild_route(
    payload=CBRMessage(routing_key="topic_2", body="This is a message for topic 1"),
    topic=guild_default_topic,
)

In [ ]:
probe_agent.print_message_history()